# Brain-Heart Coupling (SDG) Translation

This notebook translates the MATLAB function `model_psv_sdg` into Python.

It estimates directional couplings:
- Heart→Brain: ARX coefficient of EEG band power with CSI/CVI as exogenous inputs.
- Brain→Heart: Normalized averages of Cs/Cp (sympatho/vagal components) divided by local EEG power.

Core steps:
1. Compute Cs and Cp from IBI windows using a simplified Poincaré-based dynamic model.
2. Align time series (EEG power, CSI, CVI, Cs, Cp) to a common overlapping time axis via interpolation.
3. For each EEG channel, run SDG:
   - Sliding window ARX regression extracting the exogenous input coefficient (u coefficient).
   - Normalized Cs/Cp averages for Brain→Heart coupling.
4. Produce time vectors for each direction.

Implementation notes:
- ARX model with orders [1 1 1] in MATLAB: y(t) = a1*y(t-1) + b1*u(t-1) + c. We recover b1 via linear regression.
- MATLAB indexed `model_eegP.B(2)` (delay shift). We replicate by returning the coefficient on u(t-1).
- Interpolation uses cubic (if SciPy available) else linear.
- `clean_artif` is a placeholder (no-op) you can customize.

Below you'll find: imports, helper functions, main function, SDG subroutine, synthetic data demo, and inspection.


In [ ]:
# Imports
import numpy as np
try:
    from scipy.interpolate import CubicSpline
    _HAS_SCIPY = True
except ImportError:
    _HAS_SCIPY = False

import warnings
warnings.filterwarnings('ignore')

__all__ = [
    'model_sdg_psv'
]


In [ ]:
# Helper functions

def arx_b_coefficient(y: np.ndarray, u: np.ndarray) -> float:
    """Estimate ARX(1,1,1) coefficient for exogenous input.
    Model: y_t = a1*y_{t-1} + b1*u_{t-1} + c + e_t
    Returns b1.
    """
    if y.ndim != 1 or u.ndim != 1:
        raise ValueError("y and u must be 1D arrays")
    if len(y) != len(u):
        raise ValueError("y and u length mismatch")
    if len(y) < 3:
        return np.nan
    Y = y[1:]
    Y_lag = y[:-1]
    U_lag = u[:-1]
    X = np.column_stack([Y_lag, U_lag, np.ones_like(Y_lag)])
    try:
        coeffs, *_ = np.linalg.lstsq(X, Y, rcond=None)
        b1 = coeffs[1]
    except np.linalg.LinAlgError:
        b1 = np.nan
    return float(b1)

def interpolate_series(x_time: np.ndarray, x: np.ndarray, new_time: np.ndarray) -> np.ndarray:
    """Interpolate x onto new_time using cubic if SciPy available, else linear."""
    if len(x_time) < 2:
        return np.full_like(new_time, np.nan, dtype=float)
    if _HAS_SCIPY and len(x_time) >= 4:
        cs = CubicSpline(x_time, x)
        return cs(new_time)
    return np.interp(new_time, x_time, x)

def clean_artif(eeg: np.ndarray) -> np.ndarray:
    """Placeholder artifact cleaning (no-op). Replace with custom logic if needed."""
    return eeg


In [ ]:
# Main translation
from typing import Tuple

def model_sdg_psv(EEG_comp: np.ndarray,
                  IBI: np.ndarray,
                  t_IBI: np.ndarray,
                  CSI: np.ndarray,
                  CVI: np.ndarray,
                  Fs: float,
                  time: np.ndarray,
                  wind: float) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Python translation of MATLAB model_psv_sdg.

    Inputs:
    - EEG_comp: (channels x time) EEG band-power aligned to `time`.
    - IBI: raw inter-beat intervals (seconds) (non-interpolated)
    - t_IBI: timestamps for each IBI sample
    - CSI, CVI: time-aligned indices
    - Fs: sampling rate (Hz)
    - time: time vector (seconds)
    - wind: window length (seconds)

    Returns:
    - CSI2B, CVI2B: (channels x T1) Heart→Brain ARX coefficients over time
    - B2CSI, B2CVI: (channels x T2) Brain→Heart normalized indices over time
    - tH2B, tB2H: time axes for the two coupling directions
    """
    EEG_comp = np.asarray(EEG_comp, dtype=float)
    if EEG_comp.ndim != 2:
        raise ValueError("EEG_comp must be 2D (channels x time)")
    Nch, Nt = EEG_comp.shape
    if Nt == 1:
        EEG_comp = EEG_comp.T
        Nch, Nt = EEG_comp.shape

    IBI = np.asarray(IBI, dtype=float)
    t_IBI = np.asarray(t_IBI, dtype=float)
    CSI = np.asarray(CSI, dtype=float)
    CVI = np.asarray(CVI, dtype=float)
    time = np.asarray(time, dtype=float)

    # HRV sub-model parameters
    f_lf = 0.1
    f_hf = 0.25
    w_lf = 2*np.pi*f_lf
    w_hf = 2*np.pi*f_hf

    window = int(round(wind * float(Fs)))  # samples on the `time` base
    if window < 2:
        raise ValueError("window (wind*Fs) must be >= 2 samples")

    ss = window
    sc = 1
    nt = int(np.ceil((len(time) - ss) / sc))
    nt = max(nt, 1)

    Cs = np.zeros(nt, dtype=float)
    Cp = np.zeros(nt, dtype=float)
    TM = np.zeros(nt, dtype=float)

    # Compute Cs, Cp across windows along `time`
    for i in range(nt):
        ix1 = i*sc
        ix2 = ix1 + ss - 1
        ix2 = min(ix2, len(time)-1)
        ixm = int(np.floor((ix1 + ix2)/2))
        t1 = time[ix1]
        t2 = time[ix2]
        mask = (t_IBI >= t1) & (t_IBI <= t2)
        if not np.any(mask):
            Cs[i] = np.nan
            Cp[i] = np.nan
            TM[i] = time[ixm]
            continue
        I = IBI[mask]
        mu_ibi = float(np.mean(I))
        if mu_ibi <= 0 or not np.isfinite(mu_ibi):
            Cs[i] = np.nan
            Cp[i] = np.nan
            TM[i] = time[ixm]
            continue
        mu_hr = 1.0 / mu_ibi
        # guard sin denominators
        s_hf = np.sin(w_hf/(2*mu_hr))
        s_lf = np.sin(w_lf/(2*mu_hr))
        if np.isclose(s_hf, 0.0) or np.isclose(s_lf, 0.0):
            Cs[i] = np.nan
            Cp[i] = np.nan
            TM[i] = time[ixm]
            continue
        G = s_hf - s_lf
        M_11 = s_hf*w_lf*mu_hr/(s_lf*4)
        M_12 = -np.sqrt(2)*w_lf*mu_hr/(8*s_lf)
        M_21 = -s_lf*w_hf*mu_hr/(s_hf*4)
        M_22 = np.sqrt(2)*w_hf*mu_hr/(8*s_hf)
        M = np.array([[M_11, M_12],[M_21, M_22]], dtype=float)
        L = float(np.max(I) - np.min(I))
        dI = np.abs(np.diff(I))
        W = float(np.sqrt(2) * (np.max(dI) if dI.size > 0 else 0.0))
        C = (1.0/G) * (M @ np.array([L, W], dtype=float))
        Cs[i] = C[0]
        Cp[i] = C[1]
        TM[i] = time[ixm]

    # Remove rows where TM is not finite
    good = np.isfinite(TM)
    TM = TM[good]
    Cs = Cs[good]
    Cp = Cp[good]

    # Interpolate to common time axis
    t1 = max(time[0], TM[0]) if TM.size > 0 else time[0]
    t2 = min(time[-1], TM[-1]) if TM.size > 0 else time[-1]
    mask_time = (time >= t1) & (time <= t2)
    time2 = time[mask_time]
    if time2.size < 3:
        raise ValueError("Insufficient overlapping time after alignment")

    Cs_i = interpolate_series(TM, Cs, time2) if TM.size > 1 else np.full_like(time2, np.nan)
    Cp_i = interpolate_series(TM, Cp, time2) if TM.size > 1 else np.full_like(time2, np.nan)
    CSI_i = interpolate_series(time, CSI, time2)
    CVI_i = interpolate_series(time, CVI, time2)

    EEG_old = EEG_comp[:, mask_time]
    EEG_new = np.empty((Nch, time2.size), dtype=float)
    for ch in range(Nch):
        EEG_new[ch] = interpolate_series(time[mask_time], EEG_old[ch], time2)

    time_al = time2
    EEG_new = clean_artif(EEG_new)

    # SDG per channel
    CSI2B = []
    CVI2B = []
    B2CSI = []
    B2CVI = []
    for ch in range(Nch):
        csi2b, cvi2b, b2csi, b2cvi = SDG(EEG_new[ch], CSI_i, CVI_i, Cs_i, Cp_i, window)
        CSI2B.append(csi2b)
        CVI2B.append(cvi2b)
        B2CSI.append(b2csi)
        B2CVI.append(b2cvi)

    CSI2B = np.vstack([np.asarray(x, dtype=float) for x in CSI2B])
    CVI2B = np.vstack([np.asarray(x, dtype=float) for x in CVI2B])
    B2CSI = np.vstack([np.asarray(x, dtype=float) for x in B2CSI])
    B2CVI = np.vstack([np.asarray(x, dtype=float) for x in B2CVI])

    # Time vectors
    if time_al.size <= window:
        raise ValueError("Aligned time shorter than window")
    tH2B = time_al[:time_al.size - window]
    tB2H = time_al[window: time_al.size - window]

    return CSI2B, CVI2B, B2CSI, B2CVI, tH2B, tB2H


def SDG(EEG_ch: np.ndarray,
        HRV_CSI: np.ndarray,
        HRV_CVI: np.ndarray,
        Cs_i: np.ndarray,
        Cp_i: np.ndarray,
        window: int):
    Nt = EEG_ch.size
    # Initialize outputs with NaNs up to first window
    CSI_to_EEG = np.full(Nt, np.nan, dtype=float)
    CVI_to_EEG = np.full(Nt, np.nan, dtype=float)
    pow_eeg = np.full(Nt, np.nan, dtype=float)

    end_first = min(window, Nt-1)
    for i in range(0, end_first):
        j2 = min(i+window, Nt-1)
        y = EEG_ch[i:j2+1]
        u_csi = HRV_CSI[i:j2+1]
        u_cvi = HRV_CVI[i:j2+1]
        CSI_to_EEG[i] = arx_b_coefficient(y, u_csi)
        CVI_to_EEG[i] = arx_b_coefficient(y, u_cvi)
        pow_eeg[i] = float(np.mean(y))

    # Main loop
    limit = min(len(Cp_i), Nt - window, len(HRV_CSI) - window)
    for i in range(window, max(limit, window)):
        j2 = min(i+window, Nt-1)
        y = EEG_ch[i:j2+1]
        u_csi = HRV_CSI[i:j2+1]
        u_cvi = HRV_CVI[i:j2+1]
        CSI_to_EEG[i] = arx_b_coefficient(y, u_csi)
        CVI_to_EEG[i] = arx_b_coefficient(y, u_cvi)
        pow_eeg[i] = float(np.mean(y))

    # Brain→Heart normalized averages
    # Build outputs aligned like MATLAB: index starting at i-window
    max_len = max(0, limit - window)
    EEG_to_CVI = np.full(max_len if max_len>0 else 1, np.nan, dtype=float)
    EEG_to_CSI = np.full(max_len if max_len>0 else 1, np.nan, dtype=float)

    for k, i in enumerate(range(window, limit)):
        start = i - window
        end = i
        denom = pow_eeg[start:end+1]
        denom = np.where(denom == 0, np.nan, denom)
        EEG_to_CVI[k] = np.nanmean(Cp_i[start:end+1] / denom)
        EEG_to_CSI[k] = np.nanmean(Cs_i[start:end+1] / denom)
        # hold last if NaN
        if k>0:
            if not np.isfinite(EEG_to_CVI[k]):
                EEG_to_CVI[k] = EEG_to_CVI[k-1]
            if not np.isfinite(EEG_to_CSI[k]):
                EEG_to_CSI[k] = EEG_to_CSI[k-1]

    return CSI_to_EEG, CVI_to_EEG, EEG_to_CSI, EEG_to_CVI


In [ ]:
# Synthetic data demo setup
np.random.seed(42)

# Parameters
Fs = 4.0
wind = 15.0  # seconds
T_sec = 300.0  # total duration
Nt = int(T_sec * Fs)

time = np.arange(Nt) / Fs
Nch = 3  # channels

# Generate synthetic EEG band power (slow varying + noise)
EEG_comp = []
for ch in range(Nch):
    base = 1 + 0.2*ch + 0.5*np.sin(2*np.pi*0.01*time)  # slow drift
    noise = 0.1*np.random.randn(Nt)
    EEG_comp.append(base + noise)
EEG_comp = np.vstack(EEG_comp)

# Synthetic heart rate / IBI
hr_mean = 1.1  # Hz (~66 bpm)
t_IBI = [0.0]
IBI = []
while t_IBI[-1] < T_sec:
    interval = np.random.normal(1/hr_mean, 0.05)
    if interval <= 0:
        interval = 1/hr_mean
    IBI.append(interval)
    t_IBI.append(t_IBI[-1] + interval)
IBI = np.array(IBI)
t_IBI = np.array(t_IBI[:-1])  # last timestamp beyond T_sec removed

# Construct synthetic CSI/CVI trends
CSI = 2 + 0.3*np.sin(2*np.pi*0.005*time) + 0.1*np.random.randn(Nt)
CVI = 1.5 + 0.2*np.cos(2*np.pi*0.007*time) + 0.1*np.random.randn(Nt)

print(f"Synthetic data generated: EEG_comp shape={EEG_comp.shape}, IBI count={IBI.size}, CSI shape={CSI.shape}")


In [ ]:
# Run model and inspect
CSI2B, CVI2B, B2CSI, B2CVI, tH2B, tB2H = model_sdg_psv(
    EEG_comp=EEG_comp,
    IBI=IBI,
    t_IBI=t_IBI,
    CSI=CSI,
    CVI=CVI,
    Fs=Fs,
    time=time,
    wind=wind
)

print("Outputs:")
print("CSI2B shape:", CSI2B.shape)
print("CVI2B shape:", CVI2B.shape)
print("B2CSI shape:", B2CSI.shape)
print("B2CVI shape:", B2CVI.shape)
print("tH2B length:", tH2B.size, "tB2H length:", tB2H.size)
print("Sample CSI2B[0,:5]", CSI2B[0,:5])
print("Sample B2CSI[0,:5]", B2CSI[0,:5])
